In [122]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import re

# Reading in my poker tournament data
df_poker = pd.read_csv("Resources/Jamie_Poker.csv")
# Setting focus user
user = 'JJin91'

# Hand history df
df_hh = pd.DataFrame(columns=['Tournament_ID','Hand_Count','Players_At_Table','User_Position','Cards','Chip_Count'])
df_poker

,Tournament_ID,Date,Buy_In,Prize_Pool,Placement,Entrants,Award
0,1597683884,2017/05/02 19:00:00 ET,20,1983.80,7,109,79.35
1,1597684008,2017/05/05 13:30:00 ET,15,1431.00,8,106,46.50
2,1597684021,2017/05/05 19:00:00 ET,20,2093.00,1,115,481.46
3,1597684053,2017/05/06 13:30:00 ET,15,1444.50,15,107,18.77
4,1597684117,2017/05/07 21:00:00 ET,5,500.00,5,16,32.50
5,1622537113,2017/05/09 18:00:00 ET,10,1046.50,17,115,21.45
6,1622537329,2017/05/14 13:30:00 ET,15,1606.50,10,119,34.53
7,1622573245,2017/05/24 21:00:00 ET,5,541.45,14,119,11.64
8,1622573280,2017/05/25 18:00:00 ET,10,1528.80,13,168,22.93
9,1648956582,2017/06/19 19:00:00 ET,20,2875.60,13,158,43.13


In [123]:
# Position finder function based on button position, user position, and number of seats
def position_finder(button_seat_position,user_seat_position,num_of_players):
    if num_of_players == 2:
        if button_seat == user_seat:
            return 'SB'
        else:
            return 'BB'
    else:
        if button_seat == user_seat:
            return 'BT'
        elif button_seat_position - user_seat_position == -1 or button_seat_position - user_seat_position == num_of_players-1:
            return 'SB'
        elif button_seat_position - user_seat_position == -2 or button_seat_position - user_seat_position == num_of_players-2:
            return 'BB'
        elif button_seat_position - user_seat_position == -3 or button_seat_position - user_seat_position == num_of_players-3:
            return 'UTG'
        elif button_seat_position - user_seat_position == 1 or button_seat_position - user_seat_position == -(num_of_players-1):
            return 'CO'
        elif button_seat_position - user_seat_position == -4 or button_seat_position - user_seat_position == num_of_players-4:
            return 'UTG+1'
        elif button_seat_position - user_seat_position == 2 or button_seat_position - user_seat_position == -(num_of_players-2):
            return 'HJ'
        elif button_seat_position - user_seat_position == -5 or button_seat_position - user_seat_position == num_of_players-5:
            return 'MP'
        elif button_seat_position - user_seat_position == -6 or button_seat_position - user_seat_position == num_of_players-6:
            return 'MP+1'
        else:
            return 'table position error'

In [124]:
tourny = []
# Reading all hand history text files
for i in range(len(df_poker)): 
    tourny_temp = open("Resources/{}.txt".format(df_poker['Tournament_ID'].iloc[i]),'r').read()
    # Removing bottom contact message
    tourny_temp = tourny_temp.split("\n\n\n\n", 1)[0]
    # Splitting by hands and removing top hand history request message
    tourny_temp_split = re.split('\*{11}\s\#\s\d*\s\*{14}',tourny_temp)[1:]
    tourny_temp_split
    tourny.append(tourny_temp_split)

index = 0
for i in range(len(tourny)):
    hand_count = 1
    for ii in range(len(tourny[i])):
        # Tourny ID to df_hh
        tourny_id = re.search("Tournament #\d{10}", tourny[i][ii]).group().split(' ')[1][1:]
        df_hh.set_value(index, 'Tournament_ID', tourny_id)
        
        seat_list = [int(i.split(" ")[1]) for i in re.findall("Seat\s\d",tourny[i][ii].split("*** SUMMARY ***")[1])]   
        button_seat = int(re.search("Seat \#\d is the button", tourny[i][ii]).group().split(" #")[1].split(" ")[0])
        user_seat = int(re.search("Seat \d\: " + user, tourny[i][ii]).group().split(": ")[0].split(" ")[1])
        
        # Players at table of tourny[i] hand[ii] to df_hh
        df_hh.set_value(index, 'Players_At_Table', len(seat_list))
        
        # Find user position using above position finder funciton
        try:
            user_position = position_finder(seat_list.index(button_seat),seat_list.index(user_seat),len(seat_list))
            df_hh.set_value(index, 'User_Position', user_position)
        except:
            print("tournament {} hand {} position fail".format(df_poker['Tournament_ID'].iloc[i],hand_count))

        # Hand number of tourny[i] hand[ii] to df_hh
        df_hh.set_value(index, 'Hand_Count', hand_count)
        
        # User cards to df_hh
        try:
            df_hh.set_value(index, 'Cards', re.search('(?<={}\s\[)\w\w\s\w\w'.format(user),tourny[i][ii]).group().split(' '))
        except:
            print("tournament {} hand {} cards fail".format(df_poker['Tournament_ID'].iloc[i],hand_count))
        
        # Chip count before start of hand to df_hh
        df_hh.set_value(index, 'Chip_Count', int(re.search('(?<=Seat \d\: {}\s\()\d*'.format(user),tourny[i][ii]).group()))
        
        index += 1
        hand_count += 1
        
df_hh

tournament 1622537113 hand 94 position fail
tournament 1622537113 hand 94 cards fail
tournament 1648956862 hand 2 position fail
tournament 1648956862 hand 2 cards fail


,Tournament_ID,Hand_Count,Players_At_Table,User_Position,Cards,Chip_Count
0,1597683884,1,8,HJ,"[4d, 5c]",10000
1,1597683884,2,8,MP,"[3s, 8h]",9960
2,1597683884,3,8,UTG+1,"[Jd, 6h]",9910
3,1597683884,4,8,UTG,"[5s, Qs]",9860
4,1597683884,5,7,BB,"[9d, Js]",9810
5,1597683884,6,8,SB,"[Ts, 9h]",9360
6,1597683884,7,8,BT,"[Ts, 7s]",9110
7,1597683884,8,8,CO,"[9h, Ts]",18920
8,1597683884,9,8,HJ,"[5c, Qc]",18870
9,1597683884,10,7,UTG+1,"[6d, Kh]",18820


In [125]:
re.search('\d\:\s{}'.format(user),tourny[0][0]).group()
#tourny[0][0]

'8: JJin91'